## Maps

This section covers the two mapping helpers used to make plan-view figures from FVCOM–ERSEM output:

* **`domain_map`** — plots variables over the **full model domain**.
* **`region_map`** — same as above, but **masked to polygon regions** you define (shapefile or CSV boundary).

Both functions:

* accept native tracer names **or** composite/group names (from `GROUPS`),
* handle **surface / bottom / depth-averaged / sigma / fixed-z** depth selections,
* can render a **time mean** over a selected window **or** **specific instants**,
* auto-pick colour limits robustly (or use your styles/limits),
* save figures under:
  `FIG_DIR/<basename(BASE_DIR)>/maps/`

---

#### Choosing parameters

* **Variables**: any native variable in `ds` or a group from `GROUPS` (e.g., `chl`, `phyto`, `DOC`). Functions can accept multiple variables
* **Time**: choose a **window** (months/years/date range) for a mean map, or specify **instant(s)** to plot.
* **Depth**: `"surface"`, `"bottom"`, `"depth_avg"`, a **sigma index/value**, or a **fixed depth** (meters, negative downward).
* **Styling**: set colormap, normalization (e.g., `LogNorm` for positive skew), robust quantile limits, mesh overlay, figure size/DPI via `PLOT_STYLES` or function kwargs.
* **Output**: filenames encode the scope, variable, depth tag, and time label, e.g.
  `<prefix>__Map-Domain__<Var>__<DepthTag>__<TimeLabel>__Mean.png` or `__Instant.png`.

---

#### Domain vs Region maps

* **Domain maps**: plot the whole grid using the triangulation built from `lon`, `lat`, and connectivity.
* **Region maps**: apply a polygon mask **before** plotting so colour limits and statistics reflect **only in-region** values.

  * Regions come from `(name, spec)` entries in `REGIONS`, using either a **shapefile** (optionally filtered by an attribute) or a **CSV boundary** (lon/lat columns, optional convex hull).








In [2]:
#Setup

BASE_DIR = "/data/proteus1/scratch/yli/project/lake_erie/output_updated_river_var"
FILE_PATTERN = "erie_00??.nc"
FIG_DIR      = "/data/proteus1/scratch/moja/projects/Lake_Erie/fvcomersem-viz/examples/plots/"


STATIONS = [
    ("WE12", 41.90, -83.10),
    ("WE13", 41.80, -83.20),
]

REGIONS = [
    ("Central", {
        "shapefile": "../data/shapefiles/central_basin_single.shp"
    }),
    ("East", {
        "shapefile": "../data/shapefiles/east_basin_single.shp"
    }),
    ("West", {
        "shapefile": "../data/shapefiles/west_basin_single.shp"
    }),
]

GROUPS = {
    "DOC":   "R1_c + R2_c + R3_c + T1_30d_c + T2_30d_c",  # dissolved organic carbon (sum of pools)
    "phyto": ["P1_c", "P2_c", "P4_c", "P5_c"],            # total phytoplankton carbon (sum)
    "zoo":   ["Z4_c", "Z5_c", "Z6_c"],                    # total zooplankton carbon (sum)
    "chl":   "P1_Chl + P2_Chl + P4_Chl + P5_Chl",         # total chlorophyll (sum)
}

PLOT_STYLES = {
    "temp":   {"line_color": "lightblue", "cmap": "coolwarm"},
    "DOC":   {"line_color": "blue", "cmap": "viridis"},
    "chl":   {"line_color": "lightgreen", "cmap": "Greens", "vmin": 0.0, "vmax": 5.0},
    "phyto": {"line_color": "darkgreen","cmap": "YlGn"},
    "zoo":   {"line_color": "purple","cmap": "PuBu"},
}



from fvcomersemviz.io import load_from_base
from fvcomersemviz.utils import out_dir, file_prefix
from fvcomersemviz.plot import (
    hr, info, bullet, kv,
    try_register_progress_bar,
    list_files, summarize_files,
    plot_call,
    print_dataset_summary,
    ensure_paths_exist,
    sample_output_listing,
)

import matplotlib.pyplot as plt
from IPython.display import display

bullet("\nStations (name, lat, lon):")
for s in STATIONS:
    bullet(f"• {s}")

bullet("\nRegions provided:")
for name, spec in REGIONS:
    bullet(f"• {name}: {spec}")
ensure_paths_exist(REGIONS)

#  Discover files
info(" Discovering files")
files = list_files(BASE_DIR, FILE_PATTERN)
summarize_files(files)
if not files:
    print("\nNo files found. Exiting.")
    sys.exit(2)

#  Load dataset
info(" Loading dataset (this may be lazy if Dask is available)")
ds = load_from_base(BASE_DIR, FILE_PATTERN)
bullet("Dataset loaded. Summary:")
print_dataset_summary(ds)

# Where figures will go / filename prefix
out_folder = out_dir(BASE_DIR, FIG_DIR)
prefix = file_prefix(BASE_DIR)
kv("Figure folder", out_folder)
kv("Filename prefix", prefix)

In [ ]:
# --- Maps examples: domain, station, region  ---



from fvcomersemviz.plots.maps import domain_map, region_map



# 1) Domain maps

# Full argument reference for domain_map(...)
# Each parameter below is annotated with what it does and accepted values.
# Renders plan-view maps over the FULL domain; saves PNGs; returns None.

# def domain_map(
#     ds: xr.Dataset,                              # Xarray Dataset with FVCOM–ERSEM output (already opened/combined)
#     variables: List[str],                        # One or more names: native vars (e.g., "temp") or composites (e.g., "chl") if provided in `groups`
#     *,                                           # Everything after this must be passed as keyword-only (safer, clearer)
#     depth: Any,                                  # Vertical selection:
#                                                  #   "surface" | "bottom" | "depth_avg"
#                                                  #   int -> sigma layer index (e.g., 5 == k=5)
#                                                  #   float in [-1, 0] -> sigma value (e.g., -0.7)
#                                                  #   other float -> absolute depth z (meters, negative downward; e.g., -8.0 == 8 m below surface)
#                                                  #   ("siglay_index", k) | ("sigma", s) | ("z_m", z)    # explicit tuple forms
#                                                  #   {"z_m": z, "zvar": "z"}                            # dict form if vertical coord has non-default name
#     months: Optional[List[int]] = None,          # Calendar months to include (1–12) across all years; e.g., [7] or [4,5,6,7,8,9,10]; None = no month filter
#     years: Optional[List[int]] = None,           # Calendar years to include; e.g., [2018] or [2018, 2019]; None = no year filter
#     start_date: Optional[str] = None,            # Inclusive start date "YYYY-MM-DD"; used with end_date; None = open start
#     end_date: Optional[str] = None,              # Inclusive end date   "YYYY-MM-DD"; used with start_date; None = open end
#     at_time: Optional[Any] = None,               # Single timestamp to render an instantaneous map; accepts str/np.datetime64/pd.Timestamp
#     at_times: Optional[Sequence[Any]] = None,    # Multiple timestamps to render multiple instantaneous maps
#     time_method: str = "nearest",                # Selection policy when matching requested instants to data: "nearest" (typical)
#     base_dir: str,                               # Path to the model run folder; used for output subfolder and filename prefix
#     figures_root: str,                           # Root directory where figures are saved (module subfolder "maps/" is created under this)
#     groups: Optional[Dict[str, Any]] = None,     # Composite definitions enabling semantic names in `variables`:
#                                                  #   {"chl": "P1_Chl + P2_Chl + P4_Chl + P5_Chl"}    # string expression evaluated in ds namespace
#                                                  #   {"phyto": ["P1_c", "P2_c", "P4_c", "P5_c"]}     # list/tuple summed elementwise
#     cmap: str = "viridis",                       # Default colormap (overridden per-variable by `styles`, if provided)
#     clim: Optional[Tuple[float, float]] = None,  # Explicit (vmin, vmax). If None, uses `styles` vmin/vmax if set, else robust quantiles
#     robust_q: Tuple[float, float] = (5, 95),     # Percentile limits (q_low, q_high) for robust autoscaling when no norm/vmin/vmax is set
#     dpi: int = 150,                              # Output resolution (dots per inch) for saved PNG
#     figsize: Tuple[float, float] = (8, 6),       # Figure size in inches (width, height)
#     shading: str = "gouraud",                    # Tri shading mode: "gouraud" (node-centered) or "flat" (face-centered forced internally)
#     grid_on: bool = False,                       # If True, overlay the triangular mesh lines on top of the map
#     verbose: bool = False,                       # If True, print progress messages (selected times, paths, etc.)
#     styles: Optional[Dict[str, Dict[str, Any]]] = None,  # Per-variable style overrides:
#                                                  #   {"chl": {"cmap": "Greens", "vmin": 0, "vmax": 5},
#                                                  #    "zoo": {"norm": LogNorm(1e-4, 1e0), "shading": "flat"}}
# ) -> None:
#     pass  # Function selects depth/time, evaluates variables/groups, chooses color limits, plots full-domain tri map(s), and SAVES PNG(s); returns None

# Output path patterns:
#   Mean over window:
#     <figures_root>/<basename(base_dir)>/maps/
#       <prefix>__Map-Domain__<VarOrGroup>__<DepthTag>__<TimeLabel>__Mean.png
#   Instantaneous at time t:
#       <prefix>__Map-Domain__<VarOrGroup>__<DepthTag>__<YYYY-MM-DDTHHMM>__Instant.png
#
# where:
#   <prefix>    = file_prefix(base_dir)
#   <DepthTag>  = derived from `depth` (Surface, Bottom, DepthAvg, SigmaK5, SigmaS0.7, Z8m, ...)
#   <TimeLabel> = derived from months/years/start_date/end_date (AllTime, Jul, 2018, 2018-04–2018-10, ...)
#
# Notes:
# - Node- vs element-centered variables are detected by presence of 'node' or 'nele' dims and plotted accordingly.
# - If a normalization (`norm`, e.g., LogNorm) is provided via `styles`, it takes precedence over `clim`/robust quantiles.
# - Absolute-depth selections use select_da_by_z(...) per variable; sigma selections use sigma coords.
# - Returns None; to view in a notebook, display saved PNGs afterward (e.g., with the gallery cell).

# Example 1: Domain mean maps at SURFACE — per-variable styles (DOC, chl, temp); July only
domain_map(
    ds=ds,
    variables=["DOC", "chl", "temp"],
    depth="surface",
    months=[7],                         # July across all years
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    groups=GROUPS,
    styles=PLOT_STYLES,                 # per-var cmap/vmin/vmax/norm
    grid_on=True,                       # draw mesh overlay
    dpi=150, figsize=(8, 6),
    verbose=False,
)

# Example 2: Domain instantaneous maps at BOTTOM (phyto) — two timestamps
domain_map(
    ds=ds,
    variables=["phyto"],
    depth="bottom",
    at_times=["2018-06-15 00:00", "2018-09-01 12:00"],  # nearest match in data
    time_method="nearest",
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    groups=GROUPS,
    styles=PLOT_STYLES,
    grid_on=False,
    dpi=150, figsize=(8, 6),
    verbose=False,
)

# Example 3: Domain mean at ABSOLUTE depth z = -8 m (phyto), Apr–Oct 2018
domain_map(
    ds=ds,
    variables=["phyto"],
    depth=-8.0,                         # absolute depth in metres (negative downward)
    years=[2018],
    months=[4,5,6,7,8,9,10],
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    groups=GROUPS,
    styles=PLOT_STYLES,
    grid_on=True,
    dpi=150, figsize=(8, 6),
    verbose=False,
)


#2) Region maps

# Full argument reference for region_map(...)
# Each parameter below is annotated with what it does and accepted values.
# Renders plan-view maps MASKED to polygon regions; saves PNGs; returns None.

# def region_map(
#     ds: xr.Dataset,                               # Xarray Dataset with FVCOM–ERSEM output (already opened/combined).
#                                                  # Must include 'lon' and 'lat' for building region masks.
#     variables: List[str],                         # One or more names: native vars (e.g., "temp") or composites (e.g., "chl") if provided in `groups`.
#     regions: List[Tuple[str, Dict[str, Any]]],    # List of (region_name, spec_dict) entries. Each spec_dict provides EXACTLY ONE polygon source:
#                                                   #   {"shapefile": "/path/to/region.shp"}                     # optional feature filter:
#                                                   #       + "name_field": "<FIELD>", "name_equals": "<VALUE>"
#                                                   #   {"csv_boundary": "/path/to/boundary.csv"}               # CSV boundary polygon
#                                                   #       + "lon_col": "lon", "lat_col": "lat"                 # column names (defaults: lon/lat)
#                                                   #       + "convex_hull": True|False                          # wrap scattered points into a hull
#                                                   #       + "sort": "auto" | None                              # attempt to order perimeter points
#     *,                                            # Everything after this must be passed as keyword-only (safer, clearer).
#     depth: Any,                                   # Vertical selection:
#                                                   #   "surface" | "bottom" | "depth_avg"
#                                                   #   int -> sigma layer index (e.g., 5 == k=5)
#                                                   #   float in [-1, 0] -> sigma value (e.g., -0.7)
#                                                   #   other float -> absolute depth z (meters, negative downward; e.g., -8.0 == 8 m below surface)
#                                                   #   ("siglay_index", k) | ("sigma", s) | ("z_m", z)          # explicit tuple forms
#                                                   #   {"z_m": z, "zvar": "z"}                                  # dict form if vertical coord has non-default name
#     months: Optional[List[int]] = None,           # Calendar months to include (1–12) across all years; e.g., [7] or [4,5,6,7,8,9,10]; None = no month filter.
#     years: Optional[List[int]] = None,            # Calendar years to include; e.g., [2018] or [2018, 2019]; None = no year filter.
#     start_date: Optional[str] = None,             # Inclusive start date "YYYY-MM-DD"; used with end_date; None = open start.
#     end_date: Optional[str] = None,               # Inclusive end date   "YYYY-MM-DD"; used with start_date; None = open end.
#     at_time: Optional[Any] = None,                # Single timestamp to render an instantaneous map; accepts str/np.datetime64/pd.Timestamp.
#     at_times: Optional[Sequence[Any]] = None,     # Multiple timestamps to render multiple instantaneous maps.
#     time_method: str = "nearest",                 # Selection policy when matching requested instants to data: "nearest" (typical).
#     base_dir: str,                                # Path to the model run folder; used for output subfolder and filename prefix.
#     figures_root: str,                            # Root directory where figures are saved (module subfolder "maps/" is created under this).
#     groups: Optional[Dict[str, Any]] = None,      # Composite definitions enabling semantic names in `variables`:
#                                                   #   {"chl": "P1_Chl + P2_Chl + P4_Chl + P5_Chl"}             # string expression evaluated in ds namespace
#                                                   #   {"phyto": ["P1_c", "P2_c", "P4_c", "P5_c"]}              # list/tuple summed elementwise
#     cmap: str = "viridis",                        # Default colormap (overridden per-variable by `styles`, if provided).
#     clim: Optional[Tuple[float, float]] = None,   # Explicit (vmin, vmax). If None, uses `styles` vmin/vmax if set, else robust in-region quantiles.
#     robust_q: Tuple[float, float]] = (5, 95),     # Percentile limits (q_low, q_high) for robust autoscaling when no norm/vmin/vmax is set.
#     dpi: int = 150,                               # Output resolution (dots per inch) for saved PNG.
#     figsize: Tuple[float, float]] = (8, 6),       # Figure size in inches (width, height).
#     shading: str = "gouraud",                     # Tri shading mode: "gouraud" (node-centered) or "flat" (face-centered forced internally).
#     grid_on: bool = False,                        # If True, overlay the triangular mesh lines on top of the map.
#     verbose: bool = False,                        # If True, print progress messages (mask-building, selected times, paths, etc.).
#     styles: Optional[Dict[str, Dict[str, Any]]] = None,  # Per-variable style overrides:
#                                                   #   {"chl": {"cmap": "Greens", "vmin": 0, "vmax": 5},
#                                                   #    "zoo": {"norm": LogNorm(1e-4, 1e0), "shading": "flat"}}
# ) -> None:
#     pass  # Function builds a region mask (nodes/elements), selects depth/time, evaluates variables/groups,
#           # chooses color limits from in-region values, plots masked tri map(s), and SAVES PNG(s); returns None.

# Output path patterns:
#   Mean over window:
#     <figures_root>/<basename(base_dir)>/maps/
#       <prefix>__Map-Region-<Name>__<VarOrGroup>__<DepthTag>__<TimeLabel>__Mean.png
#   Instantaneous at time t:
#       <prefix>__Map-Region-<Name>__<VarOrGroup>__<DepthTag>__<YYYY-MM-DDTHHMM>__Instant.png
#
# where:
#   <prefix>    = file_prefix(base_dir)
#   <Name>      = region name from `regions`
#   <DepthTag>  = derived from `depth` (Surface, Bottom, DepthAvg, SigmaK5, SigmaS0.7, Z8m, ...)
#   <TimeLabel> = derived from months/years/start_date/end_date (AllTime, Jul, 2018, 2018-04–2018-10, ...)
#
# Notes:
# - Node- vs element-centered variables are detected by presence of 'node' or 'nele' dims and plotted accordingly.
# - Region masks: nodes are m

# Example 1: Region=CENTRAL, depth-averaged mean (zoo with log norm), mesh overlay
region_map(
    ds=ds,
    variables=["zoo"],
    regions=[REGIONS[0]],               # Central
    depth="depth_avg",
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    groups=GROUPS,
    styles=PLOT_STYLES,
    grid_on=True,
    dpi=150, figsize=(8, 6),
    verbose=False,
)

# Example 2: Region=WEST, sigma selections: k=5 for DOC, s=-0.7 for chl
region_map(
    ds=ds,
    variables=["DOC"],
    regions=[REGIONS[2]],               # West
    depth=5,                            # == ("siglay_index", 5)
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    groups=GROUPS,
    styles=PLOT_STYLES,
    grid_on=False,
    dpi=150, figsize=(8, 6),
    verbose=False,
)
region_map(
    ds=ds,
    variables=["chl"],
    regions=[REGIONS[2]],               # West
    depth=-0.7,                         # == ("sigma", -0.7)
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    groups=GROUPS,
    styles=PLOT_STYLES,
    grid_on=False,
    dpi=150, figsize=(8, 6),
    verbose=False,
)

# Example 3: Region=EAST, ABSOLUTE z = -15 m, instantaneous (DOC)
region_map(
    ds=ds,
    variables=["DOC"],
    regions=[REGIONS[1]],               # East
    depth=("z_m", -15.0),               # explicit absolute depth
    at_time="2018-08-15 00:00",
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    groups=GROUPS,
    styles=PLOT_STYLES,
    grid_on=True,
    dpi=150, figsize=(8, 6),
    verbose=False,
)

print(" Map examples completed. Figures saved under:", FIG_DIR)




In [ ]:
# Show saved figures for this run (maps)

from pathlib import Path
from IPython.display import display, Image, SVG

# Build the output root from your existing config
RUN_ROOT = Path(FIG_DIR) / Path(BASE_DIR).name     # e.g. <FIG_DIR>/<basename(BASE_DIR)>
OUT_ROOT = RUN_ROOT / "maps"                 

print("Looking under:", OUT_ROOT.resolve())

if not OUT_ROOT.exists():
    print(f" Folder does not exist: {OUT_ROOT}")
else:
    # grab newest first; include PNG and SVG
    files = sorted(
        list(OUT_ROOT.rglob("*.png")) + list(OUT_ROOT.rglob("*.svg")),
        key=lambda p: p.stat().st_mtime
    )
    if not files:
        print(f"No images found under {OUT_ROOT}")
    else:
        N = 20  # how many to show
        print(f"Found {len(files)} image(s). Showing the latest {min(N, len(files))}…")
        for p in files[-N:]:
            print("•", p.relative_to(RUN_ROOT))
            if p.suffix.lower() == ".svg":
                display(SVG(filename=str(p)))
            else:
                display(Image(filename=str(p)))
